In [26]:
import json
import warnings

import numpy as np
import pandas as pd
import torch
import json
from typing import Callable, List, Optional, Dict, Union

from evidently.test_preset import DataDriftTestPreset
from evidently.test_suite import TestSuite
from tqdm import tqdm

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.options import DataDriftOptions

In [3]:
train_df = pd.read_csv('data/train_2020-2023.csv')
test_df = pd.read_csv('data/test_data.csv')

In [4]:
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])

data_drift_report.run(reference_data=train_df.sample(n=1000), current_data=train_df[:10_000].sample(n=1000),)

/home/rid/Projects/Study/Magister/Dyploma/venv/lib/python3.10/site-packages/scipy/stats/_stats_py.py:8064: RuntimeWarning: divide by zero encountered in divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/home/rid/Projects/Study/Magister/Dyploma/venv/lib/python3.10/site-packages/scipy/stats/_stats_py.py:8064: RuntimeWarning: divide by zero encountered in divide
  terms = (f_obs_float - f_exp)**2 / f_exp


In [29]:
res_dict = json.dumps({k: {k2: v[k2] for k2 in ['stattest_threshold', 'drift_score', 'drift_detected']} for k,v in data_drift_report.as_dict()['metrics'][1]['result']['drift_by_columns'].items()})

res_dict

'{"Cell ID": {"stattest_threshold": 0.05, "drift_score": 3.677996330485201e-84, "drift_detected": true}, "HR Usage Rate": {"stattest_threshold": 0.05, "drift_score": 7.709567304002036e-09, "drift_detected": true}, "Number of Available\\nTCH": {"stattest_threshold": 0.05, "drift_score": 7.224279835989447e-12, "drift_detected": true}, "Param 1": {"stattest_threshold": 0.05, "drift_score": 3.318343646488803e-15, "drift_detected": true}, "Param 2": {"stattest_threshold": 0.05, "drift_score": 1.445613070447519e-14, "drift_detected": true}, "TCH Blocking Rate, BH": {"stattest_threshold": 0.05, "drift_score": 0.26347172719864703, "drift_detected": false}, "TCH Traffic (Erl), BH": {"stattest_threshold": 0.05, "drift_score": 0.0001969751018672361, "drift_detected": true}, "DATA": {"stattest_threshold": 0.05, "drift_score": 0.0, "drift_detected": true}}'

In [30]:
res_df = pd.DataFrame([{'drift_1': res_dict, 'drift_2': res_dict, 'cell_id': 0},
 {'drift_1': res_dict, 'drift_2': res_dict, 'cell_id': 1}])
res_df

,drift_1,drift_2,cell_id
0,"{""Cell ID"": {""stattest_threshold"": 0.05, ""drif...","{""Cell ID"": {""stattest_threshold"": 0.05, ""drif...",0
1,"{""Cell ID"": {""stattest_threshold"": 0.05, ""drif...","{""Cell ID"": {""stattest_threshold"": 0.05, ""drif...",1


In [31]:
res_df.to_csv('test_results.csv')

In [32]:
res_df = pd.read_csv('test_results.csv', index_col=0)
res_df.dtypes

drift_1    object
drift_2    object
cell_id     int64
dtype: object

In [37]:
res_df['drift_1'] = res_df['drift_1'].apply(json.loads, )

In [40]:
res_df.iloc[0]['drift_1']['Param 1']

{'stattest_threshold': 0.05,
 'drift_score': 3.318343646488803e-15,
 'drift_detected': True}

In [41]:
res_df.iloc[0]['drift_1']

{'Cell ID': {'stattest_threshold': 0.05,
  'drift_score': 3.677996330485201e-84,
  'drift_detected': True},
 'HR Usage Rate': {'stattest_threshold': 0.05,
  'drift_score': 7.709567304002036e-09,
  'drift_detected': True},
 'Number of Available\nTCH': {'stattest_threshold': 0.05,
  'drift_score': 7.224279835989447e-12,
  'drift_detected': True},
 'Param 1': {'stattest_threshold': 0.05,
  'drift_score': 3.318343646488803e-15,
  'drift_detected': True},
 'Param 2': {'stattest_threshold': 0.05,
  'drift_score': 1.445613070447519e-14,
  'drift_detected': True},
 'TCH Blocking Rate, BH': {'stattest_threshold': 0.05,
  'drift_score': 0.26347172719864703,
  'drift_detected': False},
 'TCH Traffic (Erl), BH': {'stattest_threshold': 0.05,
  'drift_score': 0.0001969751018672361,
  'drift_detected': True},
 'DATA': {'stattest_threshold': 0.05,
  'drift_score': 0.0,
  'drift_detected': True}}